# Batch Job Analysis - Data Prepare - Extract from SMF 
*Note: for reference only, no input/output sample data file provided*

**This sample notebook will demonstrate how to extract Batch Job log data from SMF Type 30 record and prepare for further analytics.**

Input data file is n days of SMF Type 30 record collected on z/OS named as HLQ.T2019XXXX.SMF30:<br>
>   HLQ.T20190001.SMF30<br> 
   HLQ.T20190002.SMF30<br>
   HLQ.T20190003.SMF30<br>
   .................<br>
   

**Key Steps includes:**
1. Extract everyday's batch jobs log data from SMF Type 30 record
2. Combine n days jobs data from csv files into one data frame for analysis
3. Remove uninterested batch job records
4. Calculate some interested metrics from original data


## Step 1: Extract batch job run log data from SMF Type 30 record

**Demonstrate how to read SMF Type 30 record into dataframe  **</p>
***Note:***
>1.It should be run on WMLz Platform with MDS(Mainframe Data Service) driver installed<br>
2.Ask for your MDS administrator for ssid,username and password<br>
3.With Mainframe Data Service Studio, you could get predefined SQL statement and view for SMF Type 30 record<br>
4.Refer to SMF Type 30 document to understand every metrics 


In [ ]:

import dsdbc
CONN = dsdbc.connect(SSID='MDS_ssid', USER='MDS_user', PASSWORD='MDS_password')

import pandas as pd
import datetime 

#path variable for SMF data output path, when change to another environment, need to change it according to data file location 
SMF_DATA_PATH=r"/username/smf/"


START_TIME=datetime.datetime.now()
print("Start:",START_TIME)




FIRST_DAY=1
LAST_DAY=91



for INDEX in range(FIRST_DAY,LAST_DAY,1):
   #to read SMF Type 30 record from mainframe system, HLQ.T2019001.SMF30 to get day 001 information, 
   #write to a csv file as 'df_D0001.csv' for further merge
   #SMF record size maybe very large, recommand to start from small file, e.g. seperate by day
   #convert to csv file will make further processing fast and convinient 

    DAY_STR=str(INDEX).rjust(4,'0')
    
    SMF_FILE_PRE='SMF_03000'
    SMF_FILE_POST= "__HLQ_T2019"+DAY_STR+"_SMF30" #align to real SMF record name
    SMF_30_TABLE = SMF_FILE_PRE+SMF_FILE_POST
    SMF_FILE_PRE='SMF_03000_SMF30'
    SMF_30_ID_TABLE = SMF_FILE_PRE+'ID'+SMF_FILE_POST
    SMF_30_CAS_TABLE = SMF_FILE_PRE+'CAS'+SMF_FILE_POST
    SMF_30_PRF_TABLE = SMF_FILE_PRE+'PRF'+SMF_FILE_POST
    SMF_30_URA_TABLE = SMF_FILE_PRE+'URA'+SMF_FILE_POST
    SMF_30_SAP_TABLE = SMF_FILE_PRE+'SAP'+SMF_FILE_POST
    
    
    QUERY='select SMF30JBN As JOB_NAME,SMF30JNM AS JOB_NUM,SMF_SSI AS TYPE,SMF_SID as SYSTEM,SMF30CLS as CLASS,\
    SMF30RSD as REQUEST_D,SMF30RST REQUEST_T,SMF30STD START_D,SMF30SIT as START_T,\
    SUBSTR(SMF_TIME,1,10) AS END_D,SUBSTR(SMF_TIME,12,11) AS END_T,\
    SUBSTR(SMF_TIME,1,23) AS END_DTSTR,\
    CAS.SMF30CPT/100 as TCB_CPU_SEC,CAS.SMF30CPS/100 as SRB_CPU_SEC,\
    (SMF30CSU /160000)  * SMF30SUS  as TCB_CPU_MS,\
    (SMF30SRB /160000)  * SMF30SUS  as SRB_CPU_MS,\
    URA.SMF30TEP as EXCP,(URA.SMF30TCN/1000000)*128 as IO_CONN_SEC,URA.SMF30AIS as SSCH,\
    PRF.SMF30SRV as SERV_UNIT,PRF.SMF30CSU as TCB_UNIT,PRF.SMF30SRB as SRB_UNIT,PRF.SMF30IO as IO_UNIT,PRF.SMF30MSO as MSO_UNIT,\
    (PRF.SMF30JQT*1024/1000000) as JQ_SEC,(PRF.SMF30RQT*1024/1000000) as RQ_SEC,\
    (PRF.SMF30HQT*1024/1000000) as HQ_SEC,(PRF.SMF30SQT*1024/1000000) as SQ_SEC,\
    PRF.SMF30SCN as SERV_CLASS,\
    ID.SMF30USR as USER_NAME,\
    SAP.SMF30PGI as PAGE_IN,SAP.SMF30PGO as PAGE_OUT,SAP.SMF30NSW as PAGE_SWAP \
    FROM '+SMF_30_TABLE+' A0 \
    JOIN '+SMF_30_ID_TABLE+' ID ON A0.CHILD_KEY=ID.PARENT_KEY \
    JOIN '+SMF_30_PRF_TABLE+' PRF ON A0.CHILD_KEY=PRF.PARENT_KEY \
    JOIN '+SMF_30_CAS_TABLE+' CAS ON A0.CHILD_KEY=CAS.PARENT_KEY \
    JOIN '+SMF_30_URA_TABLE+' URA ON A0.CHILD_KEY=URA.PARENT_KEY \
    JOIN '+SMF_30_SAP_TABLE+' SAP ON A0.CHILD_KEY=SAP.PARENT_KEY \
    where A0.SMF_STY=5'
    
    #following statement will extract data from SMF record, may cost some time according to you SMF record size
    DF=pd.read_sql(QUERY,con=CONN)
    DF.to_csv(SMF_DATA_PATH +"df_D"+ DAY_STR + ".csv")
    print("After write "+"df_D"+ DAY_STR + ".csv",datetime.datetime.now())


## Step 2: Merge n days jobs data from n csv files into one file


In [ ]:

#calculation for SMF record for everyday's data, and then merge
DF = pd.DataFrame()

for INDEX in range(FIRST_DAY,LAST_DAY,1):
    SMF_30_FILE = SMF_DATA_PATH +"df_D"+ str(INDEX).rjust(4,'0') + ".csv"
    DF_1D=pd.read_csv(SMF_30_FILE, encoding='ISO-8859-1')
    DF=DF.append(DF_1D)
DF.to_csv(SMF_DATA_PATH+'df_all.csv')   

print("After write df_all.csv:",datetime.datetime.now())


## Step 3: Remove uninterested batch job records


In [ ]:

#read from the merged CSV file, 
#it could be a start point when you collect all necessary data from SMF

DF=pd.read_csv(SMF_DATA_PATH+'df_all.csv',encoding='ISO-8859-1')
print("After read df_all.csv:",datetime.datetime.now())

#only focused on batch window between 20pm to 1d+6am
DF['is_focused']=DF['START_T'].apply(lambda t:(t>=7200000) or (t<2160000))
DF=DF[DF['is_focused']==True].drop('is_focused',axis=1)
#only focused on jobs with 'JOBxxxxxx' JESID
DF['is_focused']=DF['JOB_NUM'].apply(lambda num:num[0:3]=='JOB')
DF=DF[DF['is_focused']==True].drop('is_focused',axis=1)
#remove BATCHXXX which may run several days
DF['is_focused']=DF['SERV_CLASS'].apply(lambda s:s!='BATCHXXX')
DF=DF[DF['is_focused']==True].drop('is_focused',axis=1)



## Step 4: Calculate some interested metrics from original SMF record 
   - START_DT: job start running date and time
   - END_DT: job finish date and time
   - ELAPSED_TIME: duration of job run, the second between START_DT and END_DT
   - CPU_SEC: second of CPU run time on the job
   - QUEUE_SEC: second of job waiting in various queue


In [ ]:
#translate SMF datetime to normal python datetine
import datetime, time

def SmfConv2Dt(d,t):
    YYY=int(int(d)/1000)
    DDD=int(d)-1000*YYY
    DT=datetime.datetime(YYY+1900,1,1)+datetime.timedelta(days=DDD-1)+datetime.timedelta(seconds=t/100)
    return DT


def Dt2Sec(DT):
    SEC=(DT-datetime.datetime(1970,1,1))/datetime.timedelta(seconds=1)
    return SEC


In [ ]:

DF['START_DT']=DF.apply(lambda row:SmfConv2Dt(row['START_D'],row['START_T']),axis=1)
DF['START_DTSTR']=DF['START_DT'].apply(lambda dt:datetime.datetime.strftime(dt,'%Y-%m-%d-%H.%M.%S.000000'))
DF['START_SEC']=DF['START_DT'].apply(lambda dt:Dt2Sec(dt))

DF['REQUEST_DT']=DF.apply(lambda row:SmfConv2Dt(row['REQUEST_D'],row['REQUEST_T']),axis=1)
DF['REQUEST_SEC']=DF['REQUEST_DT'].apply(lambda dt:Dt2Sec(dt))

DF['END_DT']=DF['END_DTSTR'].apply(lambda dt_str:datetime.datetime.strptime(dt_str,'%Y-%m-%d-%H.%M.%S.%f'))
DF['END_SEC']=DF['END_DT'].apply(lambda dt:Dt2Sec(dt))

DF['ELAPSED_SEC']=DF['END_SEC']-DF['START_SEC']


DF['CPU_SEC']=DF['TCB_CPU_SEC']+DF['SRB_CPU_SEC']
DF['CPU_MS']=DF['TCB_CPU_MS']+DF['SRB_CPU_MS']

DF['QUEUE_SEC']=DF['JQ_SEC']+DF['RQ_SEC']+DF['HQ_SEC']+DF['SQ_SEC']

#set job's batch_date as D-1 when START_TIME is earlier than 6am
DF['BATCH_DATE']=DF['START_DT'].apply(lambda dt:datetime.datetime.date(dt+datetime.timedelta(hours=-6)))

print(DF.shape)
print(DF.head(5))
DF.to_csv(SMF_DATA_PATH+'df_smf.csv',index=False)
print("After write df_smf.csv:",datetime.datetime.now())


In [ ]:
#print end time used for data processing
#to prepare data in large size may cost long time
END_TIME=datetime.datetime.now()
print("Finish:",END_TIME)

## Sample output of smf data extracted from previous steps
In sample dataset, df_smf is sample data extracted from previous steps <p>
Following fields are useful for further elapsed time analysis: <p>
>JOB_NAME: Job name defined by user <br>
JOB_NUM:  Job instance number <br>
START_D:  Job start date<br>
START_T:  Job start time<br>
START_DTSTR: Job start date time in string format<br>
END_D:    Job end date<br>
END_T:    Job end time<br>
END_DTSTR:   Job end date time in string format<br>
ELAPSED_SEC: Job elapsed time in second <br>

In [1]:
#import following code automatically by clicking right icon of "find data" in top toolbar
#select "df_smf.csv"-Insert Pandas DataFrame

import pandas as pd
import dsx_core_utils
from dsx_core_utils import ProjectContext
# Add asset from data set
PC = ProjectContext.ProjectContext('Batch_Job_Analytics', '1_BatchJob_SMF30Extract', '', 'xx.xx.xx.xx')
FILE_PATH = dsx_core_utils.get_local_dataset(PC, 'DF_smf.csv')
DF_DATA_1 = pd.read_csv(FILE_PATH)
DF_DATA_1.head()

,CLASS,END_D,END_DTSTR,END_T,EXCP,HQ_SEC,IO_CONN_SEC,IO_UNIT,JOB_NAME,JOB_NUM,...,START_SEC,REQUEST_DT,REQUEST_SEC,END_DT,END_SEC,ELAPSED_SEC,CPU_SEC,CPU_MS,QUEUE_SEC,BATCH_DATE
0,100,2018/8/1,2018-08-01-00.01.44.850,00.01.44.85,148,0,0,695,JN_001,JOB35880,...,1533081526,58:45.3,1533081525,01:44.8,1533081705,178.75,10,10907,2,2018/7/31
1,100,2018/8/1,2018-08-01-00.01.32.720,00.01.32.72,65147,0,27,293934,JN_002,JOB36624,...,1533081639,00:30.6,1533081631,01:32.7,1533081693,53.28,10,10747,9,2018/7/31
2,100,2018/8/1,2018-08-01-00.02.56.380,00.02.56.38,47312,0,48,190858,JN_002,JOB37496,...,1533081695,01:33.2,1533081693,02:56.4,1533081776,81.11,33,35357,2,2018/7/31
3,100,2018/8/1,2018-08-01-00.04.57.270,00.04.57.27,266848,0,335,1015707,JN_003,JOB35916,...,1533081576,58:51.7,1533081532,04:57.3,1533081897,321.32,206,218757,44,2018/7/31
4,100,2018/8/1,2018-08-01-00.02.02.810,00.02.02.81,142618,0,148,609758,JN_004,JOB35536,...,1533081503,57:57.2,1533081477,02:02.8,1533081723,220.12,88,94262,25,2018/7/31


In [2]:
DF_DATA_1.describe()

,CLASS,EXCP,HQ_SEC,IO_CONN_SEC,IO_UNIT,JQ_SEC,MSO_UNIT,PAGE_IN,PAGE_OUT,PAGE_SWAP,...,TCB_CPU_MS,TCB_CPU_SEC,TCB_UNIT,START_SEC,REQUEST_SEC,END_SEC,ELAPSED_SEC,CPU_SEC,CPU_MS,QUEUE_SEC
count,207.000000,207.000000,207.0,207.000000,2.070000e+02,207.000000,2.070000e+02,207.0,207.0,207.0,...,207.000000,207.000000,2.070000e+02,2.070000e+02,2.070000e+02,2.070000e+02,207.000000,207.000000,207.000000,207.000000
mean,172.463768,11266.067633,0.0,6.188406,5.225798e+04,0.173913,2.319452e+06,0.0,0.0,0.0,...,6541.507246,6.072464,4.495678e+06,1.533084e+09,1.533083e+09,1.533084e+09,33.151063,6.125604,6655.188406,21.768116
std,44.777957,33806.621647,0.0,30.136876,1.519144e+05,0.379954,1.235451e+07,0.0,0.0,0.0,...,22186.384600,21.032328,1.523614e+07,5.220820e+03,5.228407e+03,5.235676e+03,95.410931,21.298016,22574.096919,57.441522
min,100.000000,1.000000,0.0,0.000000,0.000000e+00,0.000000,9.000000e+00,0.0,0.0,0.0,...,0.000000,0.000000,9.170000e+02,1.533081e+09,1.533081e+09,1.533082e+09,0.000000,0.000000,0.000000,0.000000
25%,100.000000,1.000000,0.0,0.000000,0.000000e+00,0.000000,1.400000e+01,0.0,0.0,0.0,...,0.000000,0.000000,1.170000e+03,1.533082e+09,1.533082e+09,1.533082e+09,0.030000,0.000000,0.000000,0.000000
50%,200.000000,77.000000,0.0,0.000000,3.350000e+02,0.000000,4.360000e+02,0.0,0.0,0.0,...,14.000000,0.000000,1.827000e+04,1.533082e+09,1.533082e+09,1.533082e+09,0.500000,0.000000,14.000000,1.000000
75%,200.000000,1460.000000,0.0,0.000000,7.020000e+03,0.000000,1.067750e+04,0.0,0.0,0.0,...,312.500000,0.000000,2.198550e+05,1.533082e+09,1.533082e+09,1.533082e+09,5.940000,0.000000,312.500000,3.500000
max,200.000000,266848.000000,0.0,335.000000,1.015707e+06,1.000000,1.162908e+08,0.0,0.0,0.0,...,214389.000000,202.000000,1.472287e+08,1.533099e+09,1.533099e+09,1.533099e+09,600.170000,206.000000,218757.000000,288.000000
